<a href="https://colab.research.google.com/github/CantoneseCounsellorChatbot/CantoneseChatbot/blob/main/chatbot_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf /content/CantoneseChatbot
!pip install torch
!pip install simpletransformers
!pip install pytorch_pretrained_bert
!pip install zipfile
# !pip install sqlite3
!git clone https://github.com/CantoneseCounsellorChatbot/CantoneseChatbot.git
# download question retrieval model
!python /content/CantoneseChatbot/gdown2.py


In [2]:
import sys
# from google.colab import drive
# drive.mount('/content/drive')
sys.path.append('/content/CantoneseChatbot/')
sys.path.append('/content/CantoneseChatbot/GPT2')
from datetime import time
import time
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
from tqdm import tqdm
from simpletransformers.classification import ClassificationModel
from transformers import BertTokenizer, BertForSequenceClassification
# import sqlite3
import re
from models import *

In [3]:
# The default response order is advice-->question-->restatement-->bertsum, you can change the order by changing the parameter b
# The threshold of "advice","question"and"restatement" ranges from 0 to 1.1
# The threshold of "dialogpt" ranges from 0 to 20
# mode:debug or normal

chatbot_params={"advice":{"Threshold":1.0,"order":2},
                "question":{"Threshold":1.0,"order":5},
                "restatement":{"Threshold":1.0,"order":1},
                "dialogpt":{"Threshold":9,"order":10},
                "general":{"order":7},
                "mode":"debug"
                }

In [ ]:
chatbot(chatbot_params)